In [6]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE

In [7]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2024/06/19 10:30:59 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/user/GITs/Zoomcamps/MLOps/mlops-material/4-Deployment/web-service-mlflow/artifacts-local/1', creation_time=1718785859986, experiment_id='1', last_update_time=1718785859986, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [8]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"]
    df["duration"] = df["duration"].dt.total_seconds()

    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

def prepare_dictionaries(df: pd.DataFrame):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")
    return dicts

Obtaining the data from here:
- https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [10]:
df_train = read_dataframe("data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")

target = "duration"
y_train = df_train[target]
y_val = df_val[target]

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [12]:
# Important: AWS credentials have to be set e.g. with aws configure

with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs=-1)
    
    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)
    
    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)
    
    rmse = MSE(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.sklearn.log_model(model, artifact_path="model")
    
    with open("dict_vectorizer.bin", "wb") as f_out:
        pickle.dump(dv, f_out)
    
    mlflow.log_artifact("dict_vectorizer.bin")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 16.53467808612593
